In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader

from src.data import AudioDatasetInference
from src.models import BasicClassifier
from src.utils import score

/home/cedric/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_dir = 'data'
train_dir = base_dir + '/train_audio/'
test_dir = base_dir + '/test_soundscapes/'
unlabeled_dir = base_dir + '/unlabeled_soundscapes/'

class_names = sorted(os.listdir(train_dir))
n_classes = len(class_names)
class_labels = list(range(n_classes))
label2name = dict(zip(class_labels, class_names))
name2label = {v:k for k,v in label2name.items()}

In [4]:
test_df = pd.read_csv('valid_df.csv')
files = test_df['filepath']
targets = test_df['target']

In [5]:
n_classes = 182
test_dataset = AudioDatasetInference(files, targets=None, n_classes=n_classes, duration=5)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)

In [6]:
device = torch.device('cuda')

model = BasicClassifier(n_classes, pretrained=True).to(device)
checkpoint_name = "checkpoints/efficientnet_v2_s_imagenet_base_32.pth"
checkpoint = torch.load(checkpoint_name, map_location='cpu')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [10]:
ids = []
preds = np.empty(shape=(0, n_classes), dtype='float32')

test_iter = tqdm(range(len(test_dataset)))
for i in test_iter:
    specs, file = test_dataset[i]
    filename = file.split('/')[-1][:-4]
    specs = specs.to(device)
    
    with torch.no_grad():
        outs = model(specs)
        outs = nn.functional.softmax(outs, dim=1).detach().cpu()

    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(specs))]
    ids += frame_ids

    preds = np.concatenate([preds, outs], axis=0)

100%|██████████| 4892/4892 [09:30<00:00,  8.58it/s] 


In [13]:
preds.shape

(41777, 182)

,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,barswa,...,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1,target
count,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,...,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,4.177700e+04,41777.000000
mean,6.201795e-03,1.062672e-02,5.032634e-03,4.724374e-04,1.193423e-02,1.523317e-04,6.526313e-04,7.585039e-04,2.793207e-04,3.249640e-02,...,3.909016e-04,4.662952e-03,3.778348e-03,6.037107e-03,1.126869e-02,2.273498e-04,8.745489e-04,1.223989e-04,9.589118e-03,73.604711
std,1.666745e-02,2.569967e-02,2.179432e-02,3.291188e-03,3.290289e-02,1.408183e-03,2.779891e-03,6.007601e-03,1.935249e-03,7.120551e-02,...,3.213739e-03,2.277466e-02,1.621723e-02,1.861588e-02,3.493118e-02,9.412312e-04,5.407379e-03,1.137161e-03,3.294986e-02,50.455416
min,3.863898e-10,1.231771e-08,8.428349e-10,1.996736e-11,2.857773e-10,4.819793e-12,9.834163e-12,9.394703e-14,5.584176e-12,6.519280e-08,...,4.335576e-12,2.942689e-09,2.771318e-10,2.048750e-07,9.509377e-11,2.548200e-12,1.193256e-12,2.338770e-12,3.880675e-07,0.000000
25%,7.668906e-04,9.264194e-04,3.444882e-04,1.874943e-05,1.320408e-03,1.841844e-06,5.427812e-05,2.180173e-05,9.379202e-06,4.048649e-03,...,2.108786e-05,2.315370e-04,2.183181e-04,6.262859e-04,1.114765e-03,8.834500e-06,4.314990e-05,1.665278e-06,1.083554e-03,33.000000
50%,2.241388e-03,3.225964e-03,1.162751e-03,7.036459e-05,3.954360e-03,7.607206e-06,1.747055e-04,9.486527e-05,3.762457e-05,1.170743e-02,...,7.306557e-05,7.852175e-04,7.797772e-04,1.875969e-03,3.359729e-03,3.719285e-05,1.555454e-04,7.597778e-06,2.908994e-03,70.000000
75%,5.948150e-03,9.927367e-03,3.540781e-03,2.413811e-04,1.102945e-02,2.971022e-05,5.096063e-04,3.607460e-04,1.368521e-04,3.098860e-02,...,2.358265e-04,2.584768e-03,2.472342e-03,5.244429e-03,9.422971e-03,1.344772e-04,5.176723e-04,3.181732e-05,7.436871e-03,106.000000
max,6.333799e-01,8.097235e-01,9.888772e-01,2.076199e-01,9.628189e-01,8.207593e-02,2.092740e-01,5.053285e-01,1.718858e-01,9.940274e-01,...,4.724048e-01,9.706143e-01,7.958552e-01,7.115533e-01,9.793032e-01,4.059278e-02,4.231272e-01,7.246704e-02,9.819440e-01,181.000000


In [11]:
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, class_names] = preds

def get_target(row_id):
    name = row_id.split('_')[0]
    target = test_df.loc[test_df['xc_id'] == name, 'target'].values[0]
    return target

pred_df['target'] = pred_df.row_id.map(get_target)
submission = pred_df[class_names]

solution = pd.DataFrame()
for class_name in class_names:
    # Create a new column where the value is 1 if the label matches the class index, and 0 otherwise
    solution[class_name] = (pred_df['target'] == name2label[class_name]).astype(int)

/tmp/ipykernel_150400/3130283487.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred_df['target'] = pred_df.row_id.map(get_target)
/tmp/ipykernel_150400/3130283487.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solution[class_name] = (pred_df['target'] == name2label[class_name]).astype(int)
/tmp/ipykernel_150400/3130283487.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

In [12]:
score(solution, submission, 'row_id')

0.6097012204702028

In [18]:
pred_df.head(30)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1,target
0,XC756601_5,0.002630,0.015080,0.001529,0.000028,0.003712,1.806909e-06,0.000369,1.081393e-06,3.676947e-05,...,0.000568,0.000103,0.000066,0.001026,0.000126,1.861497e-04,0.000002,6.483643e-04,0.000088,139
1,XC756601_10,0.000685,0.000622,0.000238,0.000008,0.001599,1.940699e-07,0.000059,1.203048e-07,1.714421e-05,...,0.000464,0.000048,0.000015,0.000208,0.001796,2.619723e-06,0.000014,3.804449e-04,0.000062,139
2,XC756601_15,0.005079,0.000165,0.003467,0.000003,0.001357,9.642807e-08,0.000158,5.396671e-06,9.081466e-06,...,0.000261,0.000033,0.000056,0.003161,0.000308,2.353099e-05,0.000006,1.028877e-03,0.000043,139
3,XC756601_20,0.001118,0.003576,0.000853,0.000065,0.048988,3.553077e-05,0.000173,1.332497e-03,1.812702e-04,...,0.000299,0.001274,0.009416,0.008995,0.000577,2.784823e-05,0.001760,4.488797e-06,0.003070,139
4,XC756601_25,0.002985,0.001214,0.000007,0.000007,0.006100,1.708488e-06,0.000635,6.468000e-05,7.462902e-06,...,0.000001,0.000511,0.000179,0.000461,0.012366,3.464679e-06,0.000033,3.770710e-08,0.000323,139
5,XC756601_30,0.000589,0.000132,0.007702,0.000019,0.000435,2.823442e-05,0.000042,1.281502e-03,6.445240e-05,...,0.000014,0.060516,0.005159,0.001397,0.002099,5.900901e-05,0.000082,4.016891e-07,0.139044,139
6,XC756601_35,0.001406,0.003722,0.000127,0.000116,0.011800,1.117657e-05,0.000398,4.705411e-04,6.103696e-06,...,0.000027,0.000930,0.000499,0.001412,0.028810,7.426153e-06,0.000172,5.796080e-07,0.012893,139
7,XC756601_40,0.011152,0.002333,0.000159,0.000535,0.002115,1.379954e-04,0.001585,9.661235e-04,6.593070e-04,...,0.000550,0.000322,0.004594,0.010217,0.000582,3.334399e-05,0.000225,1.781803e-05,0.000723,139
8,XC756601_45,0.002321,0.007784,0.002150,0.000190,0.000850,1.698514e-05,0.000050,3.782778e-03,1.009040e-04,...,0.000032,0.001651,0.000641,0.000325,0.032090,4.532235e-05,0.001114,8.625884e-06,0.281740,139
9,XC756601_50,0.005240,0.002573,0.004790,0.001531,0.003738,7.235030e-05,0.000023,2.325403e-04,1.184422e-04,...,0.000117,0.001149,0.001144,0.004024,0.004159,5.246757e-05,0.000101,1.918984e-06,0.138511,139


In [1]:
pred_df.to_csv('test.csv', index=False)

NameError: name 'pred_df' is not defined

In [19]:
xx = pd.read_csv('test.csv')

In [20]:
xx

,Unnamed: 0,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,...,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1,target
0,0,XC756601_5,0.002630,0.015080,0.001529,0.000028,0.003712,1.806909e-06,0.000369,1.081393e-06,...,0.000568,0.000103,0.000066,0.001026,0.000126,0.000186,0.000002,6.483643e-04,0.000088,139
1,1,XC756601_10,0.000685,0.000622,0.000238,0.000008,0.001599,1.940699e-07,0.000059,1.203048e-07,...,0.000464,0.000048,0.000015,0.000208,0.001796,0.000003,0.000014,3.804449e-04,0.000062,139
2,2,XC756601_15,0.005079,0.000165,0.003467,0.000003,0.001357,9.642807e-08,0.000158,5.396670e-06,...,0.000261,0.000033,0.000056,0.003161,0.000308,0.000024,0.000006,1.028877e-03,0.000043,139
3,3,XC756601_20,0.001118,0.003576,0.000853,0.000065,0.048988,3.553077e-05,0.000173,1.332497e-03,...,0.000299,0.001274,0.009416,0.008995,0.000577,0.000028,0.001760,4.488796e-06,0.003070,139
4,4,XC756601_25,0.002985,0.001214,0.000007,0.000007,0.006100,1.708488e-06,0.000635,6.468000e-05,...,0.000001,0.000511,0.000179,0.000461,0.012366,0.000003,0.000033,3.770710e-08,0.000323,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41772,41772,XC493181_60,0.000833,0.004182,0.005688,0.000033,0.026400,3.715507e-05,0.000037,1.198244e-04,...,0.000271,0.000296,0.000095,0.000985,0.008691,0.000008,0.000429,8.151869e-05,0.000731,10
41773,41773,XC493181_65,0.004899,0.009130,0.003177,0.002091,0.013983,1.629001e-05,0.002164,5.767886e-05,...,0.000492,0.003032,0.007130,0.004210,0.002038,0.000048,0.000091,6.134300e-05,0.010749,10
41774,41774,XC493181_70,0.000252,0.002724,0.002290,0.000294,0.000147,9.447167e-06,0.000494,1.330459e-06,...,0.000032,0.026561,0.000082,0.000260,0.000071,0.000013,0.000003,5.637811e-06,0.002162,10
41775,41775,XC750334_5,0.005262,0.004812,0.000305,0.000024,0.004555,3.983036e-06,0.000415,1.541633e-05,...,0.000025,0.000185,0.000685,0.003649,0.008156,0.000012,0.000038,2.765975e-05,0.005658,106
